In [1]:
#CS221_11292018_0824 includes a RAKE class that applies RAKE on the input file
import tensorflow as tf
import pandas as pd
import json
import numpy as np
import sys
import collections
import gzip
import statistics as stat
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize 
from nltk.stem import WordNetLemmatizer

In [2]:
#reading data
        
#This block reads 5-core dataset
# number_of_data_points_to_read= 10000 #Select number of data points to read
counter= 0
data=[]
path= '/Users/srishti/Desktop/reviews_Health_and_Personal_Care_5.json'
with open(path) as f:
    for line in f:
#         if counter <number_of_data_points_to_read:
        data.append(json.loads(line))
        counter+=1
#         else: 
#             break

X=[] #reivews
Y=[] #ratings



In [5]:
number_of_asins_to_select=110

asin_mother_dict= collections.defaultdict(dict)
asin_pos_reviews_dict = collections.defaultdict(list)
temp_dict_all_parameters_known={}
# counter= len(data)
counter=0
#Number of data_points_to_select
number_of_data_points_to_select= len(data)
for i in data:
    if (len(asin_mother_dict.keys())< number_of_asins_to_select) or (i['asin'] in asin_mother_dict.keys()):
        if i['asin'] not in asin_mother_dict.keys():
            temp_dict={}
            temp_dict['pos_reviews']=""
            temp_dict['neg_reviews']=""
            asin_mother_dict[i['asin']]=temp_dict
            asin_pos_reviews_dict[i['asin']]= "" 

    #     asin_pos_reviews_dict[i['asin']].append(i['reviewText'])
        if i['overall'] >= 4:
            temp_dict={}
            temp_dict['pos_reviews']= asin_mother_dict[i['asin']]['pos_reviews']+ " "+ i['reviewText']
            asin_mother_dict[i['asin']].update(temp_dict) 
            asin_pos_reviews_dict[i['asin']]= asin_mother_dict[i['asin']]

        else:
            temp_dict={}
            temp_dict['neg_reviews']= asin_mother_dict[i['asin']]['neg_reviews']+ " "+ i['reviewText']
            asin_mother_dict[i['asin']].update(temp_dict) 
    #         asin_mother_dict[i['asin']]['neg_reviews'] = asin_mother_dict[i['asin']]['neg_reviews']+ " "+ i['reviewText']

    for i in asin_mother_dict.keys():
        if asin_mother_dict[i]['pos_reviews']=="":
            asin_mother_dict[i]['all_param_known']='False'
        else:
            asin_mother_dict[i]['all_param_known']='True'



In [6]:
counter=0
for i in asin_mother_dict.keys():
    if counter <2:
        print i+ " "+ str(asin_mother_dict[i])
        counter+=1
        

B0000533AY {'all_param_known': 'True', 'neg_reviews': u" Do not waste your money.  This thermometer only measure 1/10 of a decimal point and it is such a pain to use.  It remembers your last temperature and displays that instead of taking your current temp when you turn it on.  There is only one blue button and I have yet to figure out how to make it start recording my temp - I push the blue button once and it shows yesterday's temp, hit it again and it flashes blank but nothin else happens.  The beeping while it records (when it finally starts!) is pretty quiet and I don't really mind that.  I just wish it worked consistently and actually measured the extra decimal point.  I've been using this to help diagnose hypothyroidism, but I would not recommend this thermometer for any purpose! Takes forever to get a temperature reading. I have found much better thermometers for less money at the local drugstore. I ordered the BD brand BBT and got this lavender colored Nexcare BBT model instead

In [14]:
# asin_mother_dict_copy= asin_mother_dict.copy()


In [12]:
# asin_mother_dict= asin_mother_dict_copy.copy()


In [7]:
#This block reads meta data
#reading meta data
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

meta_data = []
# meta_path= '/Users/srishti/Desktop/meta_Books.json.gz'
meta_path= '/Users/srishti/Desktop/meta_Health_and_Personal_Care.json.gz'
counter= 1


for line in parse(meta_path):
    meta_data.append(line)

In [8]:

asin_title_map = collections.defaultdict(list)
all_asins_set= set(asin_mother_dict.keys())

for i in meta_data:
    temp_dict={}
    temp_dict_all_parameters_known={}
    if i['asin'] in all_asins_set:
        all_asins_set.remove(i['asin'])
        #Getting Titles
        if 'title' in i.keys():
            temp_dict['title']= i['title']
#             print i['asin']
#             print asin_mother_dict[i['asin']]
            asin_mother_dict[i['asin']].update(temp_dict)
    #         asin_title_map[i['asin']] = i['title']

        else:
            temp_dict['title']= None
            temp_dict['all_param_known']='False'
            asin_mother_dict[i['asin']].update(temp_dict)
            asin_title_map[i['asin']] = None

        #Adding also_bought, also_viewed, bought_together and buy_after_viewing
        temp_dict={}
        temp_dict_all_parameters_known={}
        if 'related' in i.keys():
            if 'also_bought' in i['related']:
                temp_dict['also_bought']= i['related']['also_bought']
            else:
                temp_dict['also_bought']= None

            if 'also_viewed' in i['related']:
                temp_dict['also_viewed']= i['related']['also_viewed']
            else:
                temp_dict['also_viewed']= None

            if 'bought_together' in i['related']:
                temp_dict['bought_together']= i['related']['bought_together']
            else:
                temp_dict['bought_together']= None

            if 'buy_after_viewing' in i['related']:
                temp_dict['buy_after_viewing']= i['related']['buy_after_viewing']
            else:
                temp_dict['buy_after_viewing']= None

            asin_mother_dict[i['asin']].update(temp_dict)
        else:
            temp_dict['all_param_known']='False'
            temp_dict['also_viewed']= None
            temp_dict['also_bought']= None
            temp_dict['bought_together']= None
            temp_dict['buy_after_viewing']= None

            asin_mother_dict[i['asin']].update(temp_dict)

        #Adding categories
        #NOTE HEALTH & PERSONAL CARE IS A CATEGORY FOR ALL PRODUCT, NEED TO ADDRESS THAT
        temp_dict={}
        temp_dict_all_parameters_known={}
        if 'categories' in i.keys():
            temp_dict['categories']= i['categories'][0]
            asin_mother_dict[i['asin']].update(temp_dict)
        else:
            temp_dict['categories']= None
            temp_dict['all_param_known']='False'
            asin_mother_dict[i['asin']].update(temp_dict)


            

In [9]:
count=0
for i in asin_mother_dict.keys():
    if count < 2:
        print i
        print asin_mother_dict[i]
        count+=1

B0000533AY
{'buy_after_viewing': None, 'title': '3M Digital Basal Thermometer', 'pos_reviews': u' This is a basic basal thermometer which I use for NFP. I found out about it from the book Taking Charge of Your Fertility - a system which I totally love and recommend for every girl/woman. This thermometer works well and quickly. I\'ve been using it for a few months now with no issues whatsoever. It is a good quality thermometer - and it has even stood up to my toddler\'s "attention" - when she finds it and presses the buttons a million times before I can intervene :)Very good product that can serve many purposes and definitely serves mine! It served its purpose and so far I haven\'t had any problems. It seems as accurate as possible from what I can tell. My friend recommended I get one of these and I got the best one I could find on Amazon. This works great. It\'s been really helpful so far! This is kind of loud, and if I play with the button too much in the dark, on a restart the temper

In [10]:
asin_mother_dict_copy= asin_mother_dict.copy()

In [11]:
for i in asin_mother_dict.keys():
    if asin_mother_dict[i]['all_param_known']!= 'True':
        del(asin_mother_dict[i])
print len(asin_mother_dict)

247


In [12]:
#
embeddings_index = dict()
f = open('/Users/srishti/Google Drive/000_7th Quarter/CS221/Project/untouched_data/glove.6B/glove.6B.50d.txt')
for line in f:# first element of each line is the word and remaining elements are numerical represenation of each line 
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs# creates a dict of words and numerical representation of that word
f.close()



In [13]:
embeddings_index['dog']
embeddings_index['time']

array([ 0.02648  ,  0.33737  ,  0.065667 , -0.11609  ,  0.41651  ,
       -0.21142  , -0.69582  ,  0.2822   , -0.36077  , -0.13822  ,
        0.012094 ,  0.086227 , -0.84638  ,  0.057195 ,  1.1582   ,
        0.14703  , -0.0049197, -0.24899  , -0.96014  , -0.3038   ,
        0.23972  ,  0.21058  ,  0.40608  ,  0.17789  ,  0.55253  ,
       -1.6357   , -0.17784  , -0.45222  ,  0.45805  ,  0.14239  ,
        3.7087   ,  0.40289  , -0.4083   , -0.29304  ,  0.030857 ,
       -0.15361  ,  0.10607  ,  0.63397  ,  0.12397  , -0.25349  ,
       -0.10344  ,  0.0069768, -0.17328  ,  0.35536  , -0.46369  ,
        0.15285  ,  0.41475  , -0.3398   , -0.23043  ,  0.19069  ],
      dtype=float32)

In [20]:
# -*- coding: utf-8 -*-
"""Implementation of Rapid Automatic Keyword Extraction algorithm.
As described in the paper `Automatic keyword extraction from individual
documents` by Stuart Rose, Dave Engel, Nick Cramer and Wendy Cowley.
"""

import string
from collections import Counter, defaultdict
from itertools import chain, groupby, product

import nltk
from enum import Enum
from nltk.tokenize import wordpunct_tokenize


class Metric(Enum):
    """Different metrics that can be used for ranking."""

    DEGREE_TO_FREQUENCY_RATIO = 0  # Uses d(w)/f(w) as the metric
    WORD_DEGREE = 1  # Uses d(w) alone as the metric
    WORD_FREQUENCY = 2  # Uses f(w) alone as the metric


class Rake(object):
    """Rapid Automatic Keyword Extraction Algorithm."""

    def __init__(
        self,
        stopwords=None,
        punctuations=None,
        language="english",
        ranking_metric=Metric.DEGREE_TO_FREQUENCY_RATIO,
        max_length=2,
        min_length=1,
    ):
        """Constructor.
        :param stopwords: List of Words to be ignored for keyword extraction.
        :param punctuations: Punctuations to be ignored for keyword extraction.
        :param language: Language to be used for stopwords
        :param max_length: Maximum limit on the number of words in a phrase
                           (Inclusive. Defaults to 100000)
        :param min_length: Minimum limit on the number of words in a phrase
                           (Inclusive. Defaults to 1)
        """
        # By default use degree to frequency ratio as the metric.
        if isinstance(ranking_metric, Metric):
            self.metric = ranking_metric
        else:
            self.metric = Metric.DEGREE_TO_FREQUENCY_RATIO

        # If stopwords not provided we use language stopwords by default.
        self.stopwords = stopwords
        if self.stopwords is None:
            self.stopwords = nltk.corpus.stopwords.words(language)

        # If punctuations are not provided we ignore all punctuation symbols.
        self.punctuations = punctuations
        if self.punctuations is None:
            self.punctuations = string.punctuation

        # All things which act as sentence breaks during keyword extraction.
        self.to_ignore = set(chain(self.stopwords, self.punctuations))

        # Assign min or max length to the attributes
        self.min_length = min_length
        self.max_length = max_length

        # Stuff to be extracted from the provided text.
        self.frequency_dist = None
        self.degree = None
        self.rank_list = None
        self.ranked_phrases = None

    def extract_keywords_from_text(self, text):
        """Method to extract keywords from the text provided.
        :param text: Text to extract keywords from, provided as a string.
        """
        sentences = nltk.tokenize.sent_tokenize(text)
        self.extract_keywords_from_sentences(sentences)

    def extract_keywords_from_sentences(self, sentences):
        """Method to extract keywords from the list of sentences provided.
        :param sentences: Text to extraxt keywords from, provided as a list
                          of strings, where each string is a sentence.
        """
        phrase_list = self._generate_phrases(sentences)
        self._build_frequency_dist(phrase_list)
        self._build_word_co_occurance_graph(phrase_list)
        self._build_ranklist(phrase_list)

    def get_ranked_phrases(self):
        """Method to fetch ranked keyword strings.
        :return: List of strings where each string represents an extracted
                 keyword string.
        """
        return self.ranked_phrases

    def get_ranked_phrases_with_scores(self):
        """Method to fetch ranked keyword strings along with their scores.
        :return: List of tuples where each tuple is formed of an extracted
                 keyword string and its score. Ex: (5.68, 'Four Scoures')
        """
        return self.rank_list

    def get_word_frequency_distribution(self):
        """Method to fetch the word frequency distribution in the given text.
        :return: Dictionary (defaultdict) of the format `word -> frequency`.
        """
        return self.frequency_dist

    def get_word_degrees(self):
        """Method to fetch the degree of words in the given text. Degree can be
        defined as sum of co-occurances of the word with other words in the
        given text.
        :return: Dictionary (defaultdict) of the format `word -> degree`.
        """
        return self.degree

    def _build_frequency_dist(self, phrase_list):
        """Builds frequency distribution of the words in the given body of text.
        :param phrase_list: List of List of strings where each sublist is a
                            collection of words which form a contender phrase.
        """
        self.frequency_dist = Counter(chain.from_iterable(phrase_list))

    def _build_word_co_occurance_graph(self, phrase_list):
        """Builds the co-occurance graph of words in the given body of text to
        compute degree of each word.
        :param phrase_list: List of List of strings where each sublist is a
                            collection of words which form a contender phrase.
        """
        co_occurance_graph = defaultdict(lambda: defaultdict(lambda: 0))
        for phrase in phrase_list:
            # For each phrase in the phrase list, count co-occurances of the
            # word with other words in the phrase.
            #
            # Note: Keep the co-occurances graph as is, to help facilitate its
            # use in other creative ways if required later.
            for (word, coword) in product(phrase, phrase):
                co_occurance_graph[word][coword] += 1
        self.degree = defaultdict(lambda: 0)
        for key in co_occurance_graph:
            self.degree[key] = sum(co_occurance_graph[key].values())

    def _build_ranklist(self, phrase_list):
        """Method to rank each contender phrase using the formula
              phrase_score = sum of scores of words in the phrase.
              word_score = d(w)/f(w) where d is degree and f is frequency.
        :param phrase_list: List of List of strings where each sublist is a
                            collection of words which form a contender phrase.
        """
        self.rank_list = []
        for phrase in phrase_list:
            rank = 0.0
            for word in phrase:
                if self.metric == Metric.DEGREE_TO_FREQUENCY_RATIO:
                    rank += 1.0 * self.degree[word] / self.frequency_dist[word]
                elif self.metric == Metric.WORD_DEGREE:
                    rank += 1.0 * self.degree[word]
                else:
                    rank += 1.0 * self.frequency_dist[word]
            self.rank_list.append((rank, " ".join(phrase)))
        self.rank_list.sort(reverse=True)
        self.ranked_phrases = [ph[1] for ph in self.rank_list]

    def _generate_phrases(self, sentences):
        """Method to generate contender phrases given the sentences of the text
        document.
        :param sentences: List of strings where each string represents a
                          sentence which forms the text.
        :return: Set of string tuples where each tuple is a collection
                 of words forming a contender phrase.
        """
        phrase_list = set()
        # Create contender phrases from sentences.
        for sentence in sentences:
            word_list = [word.lower() for word in wordpunct_tokenize(sentence)]
            phrase_list.update(self._get_phrase_list_from_words(word_list))
        return phrase_list

    def _get_phrase_list_from_words(self, word_list):
        """Method to create contender phrases from the list of words that form
        a sentence by dropping stopwords and punctuations and grouping the left
        words into phrases. Only phrases in the given length range (both limits
        inclusive) would be considered to build co-occurrence matrix. Ex:
        Sentence: Red apples, are good in flavour.
        List of words: ['red', 'apples', ",", 'are', 'good', 'in', 'flavour']
        List after dropping punctuations and stopwords.
        List of words: ['red', 'apples', *, *, good, *, 'flavour']
        List of phrases: [('red', 'apples'), ('good',), ('flavour',)]
        List of phrases with a correct length:
        For the range [1, 2]: [('red', 'apples'), ('good',), ('flavour',)]
        For the range [1, 1]: [('good',), ('flavour',)]
        For the range [2, 2]: [('red', 'apples')]
        :param word_list: List of words which form a sentence when joined in
                          the same order.
        :return: List of contender phrases that are formed after dropping
                 stopwords and punctuations.
        """
        groups = groupby(word_list, lambda x: x not in self.to_ignore)
        phrases = [tuple(group[1]) for group in groups if group[0]]
        return list(
            filter(
                lambda x: self.min_length <= len(x) <= self.max_length, phrases
            )
         )

In [21]:
###########################
# Helper functions
###########################

# Generate frequency dictionary of all words in all reviews (needed for TF-IDF)
def generateCorpus(X):
    all_words = defaultdict(int)
    for k in range(0,len(X)):
		wordList = re.sub("[^\w]", " ",  X[k]).split() # Clean and split data
		for words in wordList:
			all_words[words.lower()] += 1
    
    return all_words

class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t, 'n') for t in word_tokenize(doc)]

def TF_IDF_all(asin_dict):

    vectorizer = TfidfVectorizer(stop_words="english", analyzer='word', lowercase = True, tokenizer = LemmaTokenizer(), 
				 ngram_range=(1, 2), min_df = 2, max_df = 0.8)

    vec = vectorizer.fit_transform(list(asin_dict.values()))
    word_map=vectorizer.get_feature_names()

    return vec, word_map

def TF_IDF_asin(asin, asin_dict, vectorizer, word_map):
    index = asin_dict.keys().index(asin)
    tf = vectorizer[index]
    
    keywords = []
    for col in tf.nonzero()[1]:
        keywords.append((tf[0, col],word_map[col]))

    sorted_keywords = sorted(keywords, key=lambda t: t[1] * -1)

    return sorted_keywords


In [22]:
count=0
for i in asin_mother_dict.keys():
    if count< 2:
#         print asin_mother_dict[i]['pos_reviews']
        print type (asin_mother_dict[i])
#         print type (asin_mother_dict[i])
        count+=1
        

<type 'dict'>
<type 'dict'>


In [23]:
asin_dict= {}
for i in asin_mother_dict.keys():
#     print type(asin_mother_dict[i])
#     print (asin_mother_dict[i]['pos_reviews'])
    if asin_mother_dict[i]['all_param_known']=='True' :
        asin_dict[i]= asin_mother_dict[i]['pos_reviews']

number_of_asins_on_which_keyword_separators_should_be_applied= len(asin_dict.keys())
counter=0
rake_keywords_dict={}
TFIDF_keywords_dict={}
vec, map = TF_IDF_all(asin_dict)

for i in asin_dict.keys():
    if counter < number_of_asins_on_which_keyword_separators_should_be_applied:
#         print asin_dict[i]
        c = Rake()
        c.extract_keywords_from_text(asin_dict[i])
        rake_keywords_dict[i]= c.rank_list
        counter+=1
        
        ###########################
        # Run TF-IDF
        ###########################
#         #test_list = [0, 2, 3, 4, 8, 9, 12, 13, 14]
#         test_list = [0]

#         for i in test_list:
#             my_asin = asin_dict.keys()[i]
#             print my_asin
#         print asin_title_map[i]

        TFIDF_keywords_dict[i] = TF_IDF_asin(i, asin_dict, vec, map)
#             print keywords
    else:
        break
        #     rl_list.append(c.rank_list)




In [24]:
# d = Rake()
# d.extract_keywords_from_text('I love cats and cats')
# print d.rank_list
# print d.ranked_phrases
print len(TFIDF_keywords_dict.keys())


247


In [202]:
#converts reviews to embedding dict where key is the ASIN of a book
count=0
RAKE_review_dict={}
TFIDF_review_dict={}

lower_cutoff_for_word_to_be_considered=1#model parameter
TFIDF_lower_cutoff_for_word_to_be_considered=0.0#model parameter

number_of_reviews_for_clustering=100#select how many reviews are being used for clustering, small number reduces computation time. 

max_word_per_review= 20#model parameter
TFIDF_max_word_per_review= max_word_per_review#model parameter


if number_of_reviews_for_clustering > len(rake_keywords_dict.keys()):
    print ("number_of_reviews_for_clustering too large!!")
    print("number of reviews= " + str(len(rake_keywords_dict.keys()))+ " requested " + str(number_of_reviews_for_clustering))
else:
    for i in rake_keywords_dict.keys():
        normalization_factor=0
#         print i
        if count< number_of_reviews_for_clustering :
            print count
            count +=1
    #         print rake_keywords_dict[i]
            count_of_words_used=0
#             print "entering RAKE"
            for j in rake_keywords_dict[i]:
#                 print "entered RAKE"
                if count_of_words_used < max_word_per_review:
                    count_of_words_used+=1
                    if j[0] > lower_cutoff_for_word_to_be_considered:
                        if j[1] not in embeddings_index.keys():
                            for k in j[1].split():
                #                 print ("k=" + k)
                                if k in embeddings_index.keys():
                                    if i not in RAKE_review_dict.keys():

                                        RAKE_review_dict[i]= j[0]* embeddings_index[k] 
                                        normalization_factor+=1
                                    else:
                                        RAKE_review_dict[i]+= j[0]* embeddings_index[k] 
                                        normalization_factor+=1
        #                         else:
        #                             print k
                        else:
                            if j[1] in embeddings_index.keys():
                #                 print ("j=" + j[1])
                    #             RAKE_review_dict[i]+= rl_list[j][0]* embeddings_index[j][1]
                                if i not in RAKE_review_dict.keys():
                                    RAKE_review_dict[i]= j[0]* embeddings_index[j[1]]
                                    normalization_factor+=1
                                else:
                                    RAKE_review_dict[i]+= j[0]* embeddings_index[j[1]]
                                    normalization_factor+=1
        #                     else:
        #                         print j[1]
#             print ("entering normalization" + str(i))
            if normalization_factor >0:
#                     RAKE_review_dict[i]= RAKE_review_dict[i]/ normalization_factor
                [z/normalization_factor for z in RAKE_review_dict[i]]
#             print ("RAKE done..")

    #TF-IDF converts reviews to embedding dict where key is the ASIN of a book
#             print ("RAKE done..")
            TFIDF_normalization_factor=0
            TFIDF_count_of_words_used=0
#             print TFIDF_keywords_dict[i]
#             print "entering TFIDF"
            for j in TFIDF_keywords_dict[i]:
                
                if TFIDF_count_of_words_used < max_word_per_review:
                    TFIDF_count_of_words_used+=1
                    if j[0] > TFIDF_lower_cutoff_for_word_to_be_considered:
                        if j[1] not in embeddings_index.keys():
                            for k in j[1].split():
#                                 print ("k=" + k)
                                if k in embeddings_index.keys():
#                                     print ("k=" + k)
                                    if i not in TFIDF_review_dict.keys():
                                        TFIDF_review_dict[i]= j[0]* embeddings_index[k] 
                                        TFIDF_normalization_factor+=1
                                    else:
                                        TFIDF_review_dict[i]+= j[0]* embeddings_index[k] 
                                        TFIDF_normalization_factor+=1
        #                         else:
        #                             print k
                        else:
                            if j[1] in embeddings_index.keys():
#                                 print ("j=" + j[1])
                                if i not in TFIDF_review_dict.keys():
                                    TFIDF_review_dict[i]= j[0]* embeddings_index[j[1]]
                                    TFIDF_normalization_factor+=1
                                else:
                                    TFIDF_review_dict[i]+= j[0]* embeddings_index[j[1]]
                                    TFIDF_normalization_factor+=1
        #                     else:
        #                         print j[1]
#             print ("entering TFIDF Normalization" + str(i))
#             print TFIDF_review_dict
            if TFIDF_normalization_factor >0:
#                     RAKE_review_dict[i]= RAKE_review_dict[i]/ normalization_factor
                [z/TFIDF_normalization_factor for z in TFIDF_review_dict[i]]






0


/Users/srishti/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:33: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
/Users/srishti/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:36: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


/Users/srishti/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:75: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
/Users/srishti/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:89: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
/Users/srishti/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:47: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75


/Users/srishti/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:78: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [225]:
print len(TFIDF_review_dict.keys())


100


In [226]:
# count =0
# for i in RAKE_review_dict.keys():
#     if i not in TFIDF_review_dict.keys():
#         print i
        

In [227]:
RAKE_review_dict_list=[]
TFIDF_review_dict_list=[]
for i in RAKE_review_dict.keys():
    RAKE_review_dict_list.append(RAKE_review_dict[i])
    TFIDF_review_dict_list.append(TFIDF_review_dict[i])

In [228]:
#holy K-means
from sklearn.cluster import KMeans
number_of_clusters= 10
kmeans_random_state=0
RAKE_kmeans = KMeans(n_clusters=number_of_clusters, random_state=kmeans_random_state).fit_predict(RAKE_review_dict_list)
TFIDF_kmeans= KMeans(n_clusters=number_of_clusters, random_state=kmeans_random_state).fit_predict(TFIDF_review_dict_list)
print RAKE_kmeans
print TFIDF_kmeans





[8 4 6 6 4 2 6 7 3 3 1 8 4 9 6 1 7 7 6 6 1 8 8 8 6 9 6 3 1 8 5 3 3 6 6 1 0
 8 9 3 4 3 0 6 6 6 1 6 6 3 2 3 7 6 7 7 7 3 9 3 2 7 3 3 3 9 8 6 3 5 0 9 1 1
 4 6 8 3 5 5 4 8 4 3 8 4 2 6 9 6 6 6 9 5 6 3 0 1 3 8]
[1 2 2 2 6 9 2 2 7 2 2 6 2 2 8 9 2 7 7 6 8 8 2 2 6 2 3 0 2 0 3 2 7 2 8 3 2
 2 7 2 8 9 2 7 2 6 2 3 2 2 2 0 2 2 2 2 6 0 7 6 6 2 6 2 7 2 6 2 9 4 2 2 2 7
 2 2 7 7 3 5 0 3 2 2 7 7 7 8 0 2 8 7 8 6 1 6 2 2 6 2]


In [229]:
#matches the embeddings of review to a title
RAKE_clusteredTitles= collections.defaultdict(list)
RAKE_clustered_asins= collections.defaultdict(list)
for i in range(0, len(RAKE_kmeans)):
    for asin_value, word2VecList in RAKE_review_dict.items():
        if (word2VecList == RAKE_review_dict_list[i]).all():
#             print asin_value
            RAKE_clusteredTitles[RAKE_kmeans[i]].append(asin_mother_dict[asin_value]['title'])
#             temp_dict={}
#             temp_dict['asins']= RAKE_clustered_asins[RAKE_kmeans[i]].append(asin_value)
            RAKE_clustered_asins[RAKE_kmeans[i]].append(asin_value)
            break
print ("RAKE_Clustered Titles")
# print clusteredTitles

TFIDF_clusteredTitles= collections.defaultdict(list)
TFIDF_clustered_asins= collections.defaultdict(list)
for i in range(0, len(TFIDF_kmeans)):
    for asin_value, word2VecList in TFIDF_review_dict.items():
        if (word2VecList == TFIDF_review_dict_list[i]).all():
#             print asin_value
            TFIDF_clusteredTitles[TFIDF_kmeans[i]].append(asin_mother_dict[asin_value]['title'])
            TFIDF_clustered_asins[TFIDF_kmeans[i]].append(asin_value)
            break
print ("TFIDF_Clustered Titles")
# print TFIDF_clusteredTitles




RAKE_Clustered Titles
TFIDF_Clustered Titles


In [230]:
# print RAKE_clusteredTitles

In [231]:
def createOutputDicts(clustered_asins_dict):
#Create output dicts
    output_dict={}
    for i in clustered_asins_dict.keys():
        temp_dict={}
        temp_dict['asins']= clustered_asins_dict[i]
        output_dict[i]= temp_dict
    return output_dict
    # print RAKE_output_dict

# TFIDF_output_dict={}
# for i in TFIDF_clustered_asins.keys():
#     temp_dict={}
#     temp_dict['asins']= TFIDF_clustered_asins[i]
#     TFIDF_output_dict[i]= temp_dict
# print TFIDF_output_dict

In [232]:
RAKE_partial_output_dict= createOutputDicts(RAKE_clustered_asins)
TFIDF_partial_output_dict=createOutputDicts(TFIDF_clustered_asins)

In [233]:
print RAKE_partial_output_dict
print TFIDF_partial_output_dict

{0: {'asins': [u'B000056OWW', u'B00005NAXK', u'B000052X7B', u'B000066CKV']}, 1: {'asins': [u'B000068PBT', u'B00000J47L', u'B000052Y44', u'B000052YHS', u'B0000530JF', u'B00004OCL3', u'B00000JHQ6', u'B00005R19P', u'B00005T3EH']}, 2: {'asins': [u'B000068PBL', u'B000067E30', u'B000066OC7', u'B000069J4X']}, 3: {'asins': [u'3812028492', u'B000068PBJ', u'B000052YHR', u'B0000531E2', u'B000053UOC', u'B00004YK10', u'B00006AMSM', u'B00004SRAX', u'B00004YK0Y', u'B00005339L', u'B000050B6B', u'B000052YA2', u'B000052XC6', u'B00005380N', u'B000066PGS', u'B00004T7UJ', u'B00006BSXM', u'B00005T3E3', u'B00005Q50D']}, 4: {'asins': [u'B00004OCLT', u'B00004OCLS', u'B00006G9LZ', u'B000052WNY', u'B000052YBV', u'B00004Z4AE', u'B000065CIL', u'B0000530SD']}, 5: {'asins': [u'B000060NUH', u'B00000JHQ2', u'B000052YBA', u'B00005RL5E', u'B0000532GM']}, 6: {'asins': [u'B000052WWV', u'B00004OCLR', u'B0000537JP', u'B0000537JG', u'B0000645VY', u'B000052XUV', u'B00005AUID', u'B000052XUG', u'B000052YFJ', u'B000052XFF', u'B0

In [234]:
def mergetwolists(a,b):
    c=a
    for i in b:
        if i not in a:
            c.append(i)
            
    return c

In [235]:
# a=['1', '6', 'g']
# b=['e', '7', '6']

# merged_list=mergetwolists(a,b)
# print merged_list

In [236]:
def updateOutputDict(output_dict, mother_dict):
#     output_dict_copy= output_dict.copy()
    for i in output_dict:
#         print i
        temp_dict={}
        temp_dict['also_viewed']=[]
        temp_dict['also_bought']=[]
        temp_dict['bought_together']=[]
        temp_dict['buy_after_viewing']=[]
        
        for j in output_dict[i]['asins']:
#             print j
            if mother_dict[j]['also_viewed'] != None:
                if 'also_viewed' in temp_dict.keys():
                    temp_dict['also_viewed']=mergetwolists(temp_dict['also_viewed'], mother_dict[j]['also_viewed'])
                else:
                    temp_dict['also_viewed']= mother_dict[j]['also_viewed']
                    
            if mother_dict[j]['also_bought'] != None:
                if 'also_bought' in temp_dict.keys():
                    temp_dict['also_bought']=mergetwolists(temp_dict['also_bought'], mother_dict[j]['also_bought'])
                else:
                    temp_dict['also_bought']= mother_dict[j]['also_bought']

            if mother_dict[j]['bought_together'] != None:
                if 'bought_together' in temp_dict.keys():
                    temp_dict['bought_together']=mergetwolists(temp_dict['bought_together'], mother_dict[j]['bought_together'])
                else:
                    temp_dict['bought_together']= mother_dict[j]['bought_together']
            
            if mother_dict[j]['buy_after_viewing'] != None:
                if 'buy_after_viewing' in temp_dict.keys():
                    temp_dict['buy_after_viewing']=mergetwolists(temp_dict['buy_after_viewing'], mother_dict[j]['buy_after_viewing'])
                else:
                    temp_dict['buy_after_viewing']= mother_dict[j]['buy_after_viewing']
            
            if mother_dict[j]['categories'] != None:
                if 'categories' in temp_dict.keys():
                    temp_dict['categories']=mergetwolists(temp_dict['categories'], mother_dict[j]['categories'])
                else:
                    temp_dict['categories']= mother_dict[j]['categories']

        output_dict[i].update(temp_dict)
        
    return output_dict
            


In [237]:
# testing_updateOutputDict= updateOutputDict(RAKE_output_dict,asin_mother_dict)
# print testing_updateOutputDict[0]

In [238]:
RAKE_output_dict=updateOutputDict(RAKE_partial_output_dict,asin_mother_dict)
TFIDF_output_dict= updateOutputDict(TFIDF_partial_output_dict, asin_mother_dict)
# print RAKE_output_dict
# print TFIDF_output_dict


In [239]:
print RAKE_output_dict[1]['categories']

['Health & Personal Care', 'Personal Care', 'Shaving & Hair Removal', 'Electric Shavers', 'Shaver Accessories', 'Health Care', 'Allergy, Sinus & Asthma', 'Sinus Medicine', 'Medical Supplies & Equipment', 'Home Tests', 'Drug Tests', 'Breathalyzers', 'Cold Sore & Fever Blister Treatments', 'Oral Hygiene', 'Breath Fresheners', 'Tongue Brushes, Scrapers & Cleaners', 'Foot Care', 'Foot Creams & Lotions', 'Diabetes Care', 'Sharps Containers', 'Test Strips', 'Urinalysis Strips', 'Eye Care', 'Eye Drops, Lubricants & Washes', 'Pain Relievers', 'Joint & Muscle Pain Relief', 'Medications', 'Massage & Relaxation', 'Electric Massagers', 'Denture Care', 'Adhesives', 'Household Supplies', 'Manual Shaving', 'Cleaning Tools', 'Brushes', 'Household Batteries', 'AA', 'Mopping']


In [240]:
def findTopCategories(category_dict, number_of_categories_to_pick):
    list_top_counts=[]
    list_top_categories=[]
    dict_top={}
    number_of_dict_picked= number_of_categories_to_pick
    for i in category_dict.keys():
        if len(list_top_counts) < number_of_dict_picked:
            list_top_counts.append(category_dict[i])
            list_top_categories.append(i)
        else:
            min_value= min(list_top_counts)
            if  min_value < category_dict[i]:
                
#                 print i +" " + str(category_dict[i])
#                 print list_top_four_categories
                list_top_counts[list_top_counts.index(min_value)]= category_dict[i]
                for j in list_top_categories:
                    if category_dict[j]== min_value:
                        list_top_categories[list_top_categories.index(j)]= i
                        break
                        
#     print("printing selected categories in helper function: " + str(list_top_four_categories))                
    for i in list_top_categories:
        dict_top[i]= category_dict[i]
#     print("printing selected categories in helper function: " + str(dict_top_four))
    return dict_top
    
#                 print "category_dict"+ str(category_dict)
#                 print list_top_four_categories

In [241]:
def outputSummary(output_dict, mother_dict):
    output_summary_dict={}
    
    for i in output_dict:
        count_also_viewed=0
        count_also_bought=0
        count_bought_together=0
        count_buy_after_viewing=0
        asin_count_in_cluster=0
        
#         print output_dict[i]
#         print output_dict[i]['also_viewed']

        temp_dict={}
        dict_category_distribution={}
        for j in output_dict[i]['asins']:
#             print j
            asin_count_in_cluster+=1
                
            if j in output_dict[i]['also_viewed']:
                count_also_viewed+=1
                
            if j in output_dict[i]['also_bought']:
                count_also_bought+=1
                
            if j in output_dict[i]['bought_together']:
                count_bought_together+=1
                
            if j in output_dict[i]['buy_after_viewing']:
                count_buy_after_viewing+=1
            
            for k in mother_dict[j]['categories']:
    #             print k
    #             print dict_category_distribution.keys()
                if k not in dict_category_distribution.keys():
                    dict_category_distribution[k]=1
                else:
                    dict_category_distribution[k]+=1   
#         print "dict_category_distribution for cluster " + str(i)+ ":"+ str(dict_category_distribution)
        top_categories=findTopCategories(dict_category_distribution,3)
#         print "top_four_categories for cluster "+ str(i)+ ":"+ str(top_categories)

        temp_dict["also_viewed"]= count_also_viewed
        temp_dict["also_bought"]= count_also_bought
        temp_dict["bought_together"]= count_bought_together
        temp_dict["buy_after_viewing"]= count_buy_after_viewing
        temp_dict["asin_count_in_cluster"]= asin_count_in_cluster
        temp_dict["top_categories"]= top_categories
        temp_dict["count_related_products"]= count_also_bought+count_also_viewed+count_bought_together+count_buy_after_viewing

        output_summary_dict[i]=temp_dict
    return output_summary_dict

    #     print ("cluster:" + str(i)+ " "+ "also_viewed"+ str(count_also_viewed)+" "+ "also_bought"+
    #            str(count_also_bought)+" "+"bought_together"+str(count_bought_together)+" "+
    #            "buy_after_viewing"+ str(count_buy_after_viewing)+ " "+ "asin count in cluster"+ str(asin_count_in_cluster))




In [242]:
RAKE_output_summary_dict= outputSummary(RAKE_output_dict, asin_mother_dict)
TFIDF_output_summary_dict= outputSummary(TFIDF_output_dict, asin_mother_dict)

In [243]:
RAKE_output_dict[0].keys()

['also_bought',
 'also_viewed',
 'bought_together',
 'buy_after_viewing',
 'asins',
 'categories']

In [244]:
print RAKE_output_summary_dict
print TFIDF_output_summary_dict



{0: {'asin_count_in_cluster': 4, 'buy_after_viewing': 0, 'top_categories': {'Health & Personal Care': 4, 'Health Care': 3, 'Joint & Muscle Pain Relief': 2}, 'also_bought': 0, 'also_viewed': 0, 'bought_together': 0, 'count_related_products': 0}, 1: {'asin_count_in_cluster': 9, 'buy_after_viewing': 0, 'top_categories': {'Health & Personal Care': 9, 'Household Supplies': 6, 'Personal Care': 3}, 'also_bought': 1, 'also_viewed': 0, 'bought_together': 1, 'count_related_products': 2}, 2: {'asin_count_in_cluster': 4, 'buy_after_viewing': 0, 'top_categories': {'Medical Supplies & Equipment': 2, 'Health & Personal Care': 4, 'Cushions': 2}, 'also_bought': 0, 'also_viewed': 0, 'bought_together': 0, 'count_related_products': 0}, 3: {'asin_count_in_cluster': 19, 'buy_after_viewing': 0, 'top_categories': {'Health & Personal Care': 19, 'Household Supplies': 8, 'Personal Care': 8}, 'also_bought': 1, 'also_viewed': 2, 'bought_together': 1, 'count_related_products': 4}, 4: {'asin_count_in_cluster': 8, 'b

In [245]:
#Dec 13_srishti_added this part to print error summary 
f = open("07_26_pm_Error_Analysis.txt","w")
text=" Number of ASINS= " + str(number_of_reviews_for_clustering)
f.write(text + '\n')
text= " Number of clusters: " + str(number_of_clusters) + " K-Means Random state: "+ str(kmeans_random_state)
f.write(text + '\n')

text="number of ngrams used: " + str(2)
f.write(text + '\n')

text="number of keywords used: " + str(max_word_per_review)
f.write(text + '\n')





for i in RAKE_output_summary_dict.keys():
    text = "RAKE_Cluster Number: " + str(i)+ str(RAKE_output_summary_dict[i])
#     print text
    f.write(text + '\n')
    
for i in TFIDF_output_summary_dict.keys():
    text = "TFIDF_Cluster Number: " + str(i)+ str(TFIDF_output_summary_dict[i])
    f.write(text + '\n')
f.close()

In [246]:
f = open("07_26_pm_Output.txt","w")


for i in RAKE_clusteredTitles.keys():
    text = "RAKE_Cluster Number: " + str(i)+ str(RAKE_clusteredTitles[i])
#     print text
    f.write(text + '\n')
    
for i in TFIDF_clusteredTitles.keys():
    text = "TFIDF_Cluster Number: " + str(i)+ str(TFIDF_clusteredTitles[i])
    f.write(text + '\n')

f.close()